# Fe BCC

In this example the local field at the tetrahedral site(s) in bcc-Fe is calculated.

In [1]:
import numpy as np
from muesr.core.sample import Sample                   # Retains all the sample info.
from muesr.i_o.cif.cif import load_cif                 # For loading the structure from cif files
from muesr.utilities import mago_add, show_structure   # For magnetic structure description and visualization
from muesr.utilities import muon_find_equiv            # For finding and including the symmetry equivalent muon positions in the calculation 

from muesr.engines.clfc import locfield, find_largest_sphere # Does the sum and returns the local field in its various contributions

np.set_printoptions(suppress=True,precision=5)


Create a `Sample` instance and load lattice structure from a CIF file.

In [2]:
fe=Sample()                          
load_cif(fe,"./Fe.cif");

Add the muon position

In [3]:
fe.add_muon([0.50,0.25,0.0]);

and find the symmetry equivalent positions. In BCC Fe there are 12 symmetry equivalent sites for the position provided above. These are automatically added with the following command.

In [4]:
muon_find_equiv(fe);
fe

Sample status: 

 Crystal structure:           Yes
 Magnetic structure:          No
 Muon position(s):            12 site(s)
 Symmetry data:               Yes

The definition of the magnetic structure of Fe is created with `fe.new_mm()` (new magnetic model).
The experimental magnetic moment at the Fe atoms is 2.22 Bohr magneton and the propagation vector is **0** (ferromagnetic order).

In [5]:
fe.new_mm()     
fe.mm.k=np.array([0.0,0.0,0.0])
fe.mm.fc= np.array([[0.0+0.j, 0.0+0.j, 2.22+0.j],
                     [0.0+0.j, 0.0+0.j, 2.22+0.j]])

There is another way to define the magnetic structure especially useful when using the interactive session:

    mago_add(fe)


It should appear like this on the interactive screen 

    >>> mago_add(fe)
	... Propagation vector (w.r.t. conv. rec. cell): 0.0 0.0 0.0
	... Magnetic moments in bohr magnetons and cartesian coordinates.
	... Which atom? (enter for all)Fe
	... Lattice vectors:
	...   a    2.868018200000000    0.000000000000000    0.000000000000000
	...   b    0.000000000000000    2.868018200000000    0.000000000000000
	...   c    0.000000000000000    0.000000000000000    2.868018200000000
	... Atomic positions (fractional):
	...     1 Fe  0.00000000000000  0.00000000000000  0.00000000000000  55.845
	...     2 Fe  0.50000000000000  0.50000000000000  0.50000000000000  55.845
	... FC for atom 1 Fe (3 real, [3 imag]): 0.00 0.00 2.22 
	... FC for atom 2 Fe (3 real, [3 imag]): 0.00 0.00 2.22 


The following commands may be used to visualize the lattice structure, the muon position and the magnetic order with XCrysDen.

In [6]:
#show_structure(fe, [2,2,2]);

The function `find_largest_sphere` can be used to find the largest sphere with center at the muon site(s) that can be inscribed in a 100x100x100 supercell.

In [7]:
radius=find_largest_sphere(fe,[100, 100, 100])

With the following call, the local field at the muon sites is finally evaluated.
The first argument is the sample object, the second argument spcifies that a simple sum should be performed, the third argument is the supercell dimension along the three lattice vectors and the last argument is the radius of the Lorentz sphere.

In [8]:
r=locfield(fe, 's', [100, 100, 100] ,radius) 

All the local fied contributions are contained in r, the dipolar contribution in `r[i].D`, the Lorentz contribution in `r[i]`.L, the Fermi contact contribution in `r[i].C`, and the sum of all contributions in `r[i].T` where `i` runs on the muon sites.

If `r[i].ACont` (isotropic contact coupling term) is not defined `r[i].C` is zero.
For the details on the contact coupling term see the documentation (http://muesr.readthedocs.io/en/latest/ContactTerm.html)


In [9]:
B_dip=np.zeros([len(fe.muons),3])
B_Lor=np.zeros([len(fe.muons),3])
B_Cont=np.zeros([len(fe.muons),3])
B_Tot=np.zeros([len(fe.muons),3])

for i in range(len(fe.muons)):
    B_dip[i]=r[i].D
    B_Lor[i]=r[i].L
    r[i].ACont = 0.0644
    B_Cont[i]=r[i].C
    B_Tot[i]=r[i].T

As discussed in M. Schmolz et.al (Hyperfine Interactions 31 (1986) 199-204), in BCC Fe the muon jumps between the tetrahedral sites and "the field contribution at each equivalent site is either parallel or antiparallel to the magnetization of the domains" such that  B_dip(parallel)=-2B_dip(antiparallel) [...] the average of the dipolar field at these three sites vanishes"

In [10]:
print("Dipolar Field for all the 12 tetrahedral equivalent sites")
print(B_dip) 

# This is and should be same for all the equivalent sites
print("The Lorentz field is {:4.3f} {:4.3f} {:4.3f}".format(*tuple(B_Lor[0])))

print("The contact field is {:4.3f} T".format(np.linalg.norm(B_Cont[0])))

print("Dipolar average of 1 parallel site and 2 antiparallel sites is {:4.5f} T".format(np.linalg.norm(B_dip[3]+B_dip[10]+B_dip[11])))

Dipolar Field for all the 12 tetrahedral equivalent sites
[[ 0.       0.       0.26498]
 [-0.      -0.       0.26498]
 [-0.       0.      -0.52995]
 [ 0.       0.      -0.52995]
 [ 0.       0.       0.26498]
 [-0.      -0.       0.26498]
 [ 0.      -0.       0.26498]
 [-0.      -0.       0.26498]
 [ 0.       0.      -0.52995]
 [ 0.       0.      -0.52995]
 [-0.      -0.       0.26498]
 [-0.      -0.       0.26498]]
The Lorentz field is 0.000 0.000 0.731
The contact field is 1.111 T
Dipolar average of 1 parallel site and 2 antiparallel sites is 0.00000 T
